In [2]:
#!pip install --upgrade google-api-python-client

In [21]:
import numpy as np
import pandas as pd
from datetime import datetime

Once we have our credentials set up, we can now start collecting comments! We’ll first build the service for calling the YouTube API:

In [22]:
from apiclient.discovery import build

def build_service(filename):
    with open(filename) as f:
        key = f.readline()

    YOUTUBE_API_SERVICE_NAME = "youtube"
    YOUTUBE_API_VERSION = "v3"
    return build(YOUTUBE_API_SERVICE_NAME,
                 YOUTUBE_API_VERSION,
                 developerKey=key)

In [23]:
service = build_service('apikey.json') 

In [24]:
# df = pd.read_csv('archive/US_youtube_trending_data.csv')
df = pd.read_csv('ULTIMATEFINALData.csv')
df = df.tail(2000)
videoIdList = df['video_id'].tolist()
trendingDateList = df['trending_date'].tolist()
# videoIdList
# trendingDateList

In [25]:
# videoIdList = ['9bZkp7q19f0']
# videoIdList = ['3C66w5Z0ixs','M9Pmf9AB4Mo','J78aPJ3VyNs','kXLn3HkpjaA','VIUo6yapDbc','w-aidBdvZo8','uet14uf9NsE',
#                'ua4QMFQATco','SnsPZj91R7E','SsWHMAhshPQ','49Z6Mv4_WCA','nt3VVyv5pxQ','I6hswz4rIrU','W7VK4DUHvKU',
#                'W9Aen8hG20Y','BNeDH6UTmXw','6TIsR_7nrNc','gPdUslndvVI','GTp-0S82guE']
comment_count = 2000

parse for the whole list of youtube videos
	for each video
		make a service call for 100 responses of top comments
		store the data in list
		
		make the service call 9 time for 900 more responses
		store in list
		
		create dataframe of lists
		write in csv, file name comment id

In [28]:
from langdetect import detect
def fetch_comments_teratively(videoId, comment_count):
    comment, comment_id, reply_count, like_count, published_date = [], [], [], [], []
    first_response = True
#     video_trending_date = trendingDateList[videoIdList.index(videoId)]
    if (first_response):

        response = service.commentThreads().list(
            part='snippet',
            maxResults=100,
            textFormat='plainText',
            order='time',
            videoId=videoId
        ).execute()
        first_response = False
    
#     comment, comment_id, reply_count, like_count = add_data_to_lists(response, comment, comment_id, reply_count, like_count)
    for item in response['items']:
        delta = 2
        if(delta <= 3 and delta >= 0):
            if(detect(item['snippet']['topLevelComment']['snippet']['textDisplay']) == 'en'):
                comment.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
                comment_id.append(item['snippet']['topLevelComment']['id'])
                reply_count.append(item['snippet']['totalReplyCount'])
                like_count.append(item['snippet']['topLevelComment']['snippet']['likeCount'])
                published_date.append(item['snippet']['topLevelComment']['snippet']['publishedAt'])
    
#     print(len(comment))
    while(len(comment) < comment_count and len(comment) >= 100):
        try:
            if(response['nextPageToken'] != None):
#                 print(response['nextPageToken'])
                response = service.commentThreads().list(
                    part='snippet',
                    maxResults=100,
                    textFormat='plainText',
                    order='time',
                    videoId=videoId,
                    pageToken=response['nextPageToken']
                ).execute()

                for item in response['items']:
                    delta = 2
                    if(delta <= 3 and delta >= 0):
                        if(detect(item['snippet']['topLevelComment']['snippet']['textDisplay']) == 'en'):
                            comment.append(item['snippet']['topLevelComment']['snippet']['textDisplay'])
                            comment_id.append(item['snippet']['topLevelComment']['id'])
                            reply_count.append(item['snippet']['totalReplyCount'])
                            like_count.append(item['snippet']['topLevelComment']['snippet']['likeCount'])
                            published_date.append(item['snippet']['topLevelComment']['snippet']['publishedAt'])
        except:
            print("Cannot get NextPageToken for interation and Video Id", i,videoId)
            break
    print(videoId,len(comment))
    first_response = True
    return comment, comment_id, reply_count, like_count, published_date

In [1]:
for i in range(0, len(videoIdList)):
    try:
        comment, comment_id, reply_count, like_count, published_date = fetch_comments_teratively(videoIdList[i], comment_count)
        comments = pd.DataFrame(list(zip(comment, comment_id, reply_count, like_count, published_date)),
                       columns =['comment', 'comment_id', 'reply_count', 'like_count', 'published_date'])
        if not comments.empty:
            comments.to_csv(r"comment_data/"+videoIdList[i]+".csv", index=False)
    except Exception as e:
        print("Failed on this video, trying next...", videoIdList[i])
        print(e)
        continue

In [76]:
#test code
print(len(comment))

1881


In [18]:
#test code
from langdetect import detect
try:
    response = service.commentThreads().list(
                part='snippet',
                maxResults=100,
                textFormat='plainText',
                order='time',
                videoId='7wrxP2PPzLA'
            ).execute()

    response_list = []

    for item in response['items']:
#         if(detect(item['snippet']['topLevelComment']['snippet']['textDisplay']) == 'en'):
            response_list.append(item)
#             print(item)

except Exception as e:
    print(e)
# rawdata_df = pd.DataFrame(item, columns=["Commentdata"])
# rawdata_df.to_csv(r"comment_data/rawdata_comments.csv", index=False)
print(len(response_list))

100


In [9]:
!pip install langdetect

  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993242 sha256=1b44b55095456b4ece292113164d297567a0fa17d852df818d156c4f72f8827f
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\d1\c1\d9\7e068de779d863bc8f8fc9467d85e25cfe47fa5051fff1a1bb
Successfully built langdetect
